In [1]:
import requests, json, time, re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt

In [2]:
sub_all = '/top.json?t=all'
sub_gilded = '/gilded.json'
sub_hot = '.json'

In [3]:
sub_new = '/new.json'

In [4]:
# adapted code for Adi
# posts is the returned dictionary (should pass in previously ran `posts`)
# subreddit = url string 
# subpage = the page on the subreddit (should be already defined)
def sub_query(subreddit, subpage, posts=None, pages=60):
    
    #Step 0: set initial conditions
    if posts == None:
        posts = []
    else:
        posts = posts
    url = f'https://old.reddit.com/r/{subreddit}{subpage}'
    after = None

    #Step 1: build the loop and print URL
    for _ in range(pages):
        if after == None:
            current_url = url
        elif subpage == '/top.json?t=all':
            current_url = url + '&after=' + after
        else:
            current_url = url + '?count=25&after=' + after

        print(current_url)

        #Step 2a: make the requests and handle status code. Add time.sleep now (2.b)
        res = requests.get(current_url, headers={'User-agent': 'hehehehe'})

        if res.status_code != 200:
            print('Status error', res.status_code)
            break

        #Step 3: Actually deal with the data   
        current_dict = res.json()
        current_posts = [p['data'] for p in current_dict['data']['children']]
        posts.extend(current_posts)

        after = current_dict['data']['after']

        #Step 2b:
        time.sleep(0.25)

    return posts

In [5]:
news = sub_query('news', sub_all)

https://old.reddit.com/r/news/top.json?t=all
https://old.reddit.com/r/news/top.json?t=all&after=t3_aezedn
https://old.reddit.com/r/news/top.json?t=all&after=t3_7aqilp
https://old.reddit.com/r/news/top.json?t=all&after=t3_925h9r
https://old.reddit.com/r/news/top.json?t=all&after=t3_7ib4bo
https://old.reddit.com/r/news/top.json?t=all&after=t3_afiri2
https://old.reddit.com/r/news/top.json?t=all&after=t3_9zop96
https://old.reddit.com/r/news/top.json?t=all&after=t3_73n05s
https://old.reddit.com/r/news/top.json?t=all&after=t3_77iqr8
https://old.reddit.com/r/news/top.json?t=all&after=t3_a5zya6
https://old.reddit.com/r/news/top.json?t=all&after=t3_8vjnng
https://old.reddit.com/r/news/top.json?t=all&after=t3_aqy9ba
https://old.reddit.com/r/news/top.json?t=all&after=t3_b31o1f
https://old.reddit.com/r/news/top.json?t=all&after=t3_7zwi9g
https://old.reddit.com/r/news/top.json?t=all&after=t3_6wj9ey
https://old.reddit.com/r/news/top.json?t=all&after=t3_6z3755
https://old.reddit.com/r/news/top.json?t

In [6]:
news = sub_query('news', sub_hot, news)

https://old.reddit.com/r/news.json
https://old.reddit.com/r/news.json?count=25&after=t3_cbplpl
https://old.reddit.com/r/news.json?count=25&after=t3_cbnq1e
https://old.reddit.com/r/news.json?count=25&after=t3_cazztb
https://old.reddit.com/r/news.json?count=25&after=t3_cb55a8
https://old.reddit.com/r/news.json?count=25&after=t3_cbps64
https://old.reddit.com/r/news.json?count=25&after=t3_cb38ie
https://old.reddit.com/r/news.json?count=25&after=t3_cauqe4
https://old.reddit.com/r/news.json?count=25&after=t3_ca5teu
https://old.reddit.com/r/news.json?count=25&after=t3_cad3xr
https://old.reddit.com/r/news.json
https://old.reddit.com/r/news.json?count=25&after=t3_cbplpl
https://old.reddit.com/r/news.json?count=25&after=t3_cbnq1e
https://old.reddit.com/r/news.json?count=25&after=t3_cazztb
https://old.reddit.com/r/news.json?count=25&after=t3_cb55a8
https://old.reddit.com/r/news.json?count=25&after=t3_cbps64
https://old.reddit.com/r/news.json?count=25&after=t3_cb38ie
https://old.reddit.com/r/news.

In [7]:
news = sub_query('news', sub_new, news)

https://old.reddit.com/r/news/new.json
https://old.reddit.com/r/news/new.json?count=25&after=t3_cbmxnb
https://old.reddit.com/r/news/new.json?count=25&after=t3_cbi4do
https://old.reddit.com/r/news/new.json?count=25&after=t3_cbebnm
https://old.reddit.com/r/news/new.json?count=25&after=t3_cb9lrj
https://old.reddit.com/r/news/new.json?count=25&after=t3_cb5806
https://old.reddit.com/r/news/new.json?count=25&after=t3_cb0uan
https://old.reddit.com/r/news/new.json?count=25&after=t3_cas6jh
https://old.reddit.com/r/news/new.json?count=25&after=t3_cafko9
https://old.reddit.com/r/news/new.json?count=25&after=t3_ca7y9g
https://old.reddit.com/r/news/new.json?count=25&after=t3_ca0s9r
https://old.reddit.com/r/news/new.json
https://old.reddit.com/r/news/new.json?count=25&after=t3_cbmxnb
https://old.reddit.com/r/news/new.json?count=25&after=t3_cbi4do
https://old.reddit.com/r/news/new.json?count=25&after=t3_cbebnm
https://old.reddit.com/r/news/new.json?count=25&after=t3_cb9lrj
https://old.reddit.com/r/n

In [8]:
def combine_text(posts):
    return[' '.join([post['title'], post['selftext']]) for post in posts] # build this out to a function

In [9]:
nto = sub_query('nottheonion', sub_all)

https://old.reddit.com/r/nottheonion/top.json?t=all
https://old.reddit.com/r/nottheonion/top.json?t=all&after=t3_7jx5td
https://old.reddit.com/r/nottheonion/top.json?t=all&after=t3_9xrdj2
https://old.reddit.com/r/nottheonion/top.json?t=all&after=t3_7qpq3u
https://old.reddit.com/r/nottheonion/top.json?t=all&after=t3_79nb46
https://old.reddit.com/r/nottheonion/top.json?t=all&after=t3_aycaai
https://old.reddit.com/r/nottheonion/top.json?t=all&after=t3_85b4dn
https://old.reddit.com/r/nottheonion/top.json?t=all&after=t3_8pm20x
https://old.reddit.com/r/nottheonion/top.json?t=all&after=t3_btnloy
https://old.reddit.com/r/nottheonion/top.json?t=all&after=t3_69gpah
https://old.reddit.com/r/nottheonion/top.json?t=all&after=t3_97vxkb
https://old.reddit.com/r/nottheonion/top.json?t=all&after=t3_6foh0c
https://old.reddit.com/r/nottheonion/top.json?t=all&after=t3_7q11t9
https://old.reddit.com/r/nottheonion/top.json?t=all&after=t3_74p90l
https://old.reddit.com/r/nottheonion/top.json?t=all&after=t3_am1

In [10]:
nto = sub_query('nottheonion', sub_hot, nto)

https://old.reddit.com/r/nottheonion.json
https://old.reddit.com/r/nottheonion.json?count=25&after=t3_cbn7mn
https://old.reddit.com/r/nottheonion.json?count=25&after=t3_cbphzv
https://old.reddit.com/r/nottheonion.json?count=25&after=t3_cb69rd
https://old.reddit.com/r/nottheonion.json?count=25&after=t3_c9fa0x
https://old.reddit.com/r/nottheonion.json?count=25&after=t3_c8jflz
https://old.reddit.com/r/nottheonion.json?count=25&after=t3_c6kyoo
https://old.reddit.com/r/nottheonion.json?count=25&after=t3_c3vuzd
https://old.reddit.com/r/nottheonion.json?count=25&after=t3_c2zpis
https://old.reddit.com/r/nottheonion.json
https://old.reddit.com/r/nottheonion.json?count=25&after=t3_cbn7mn
https://old.reddit.com/r/nottheonion.json?count=25&after=t3_cbphzv
https://old.reddit.com/r/nottheonion.json?count=25&after=t3_cb69rd
https://old.reddit.com/r/nottheonion.json?count=25&after=t3_c9fa0x
https://old.reddit.com/r/nottheonion.json?count=25&after=t3_c8jflz
https://old.reddit.com/r/nottheonion.json?cou

In [11]:
nto = sub_query('nottheonion', sub_new, nto)

https://old.reddit.com/r/nottheonion/new.json
https://old.reddit.com/r/nottheonion/new.json?count=25&after=t3_cbo1y3
https://old.reddit.com/r/nottheonion/new.json?count=25&after=t3_cbfwl0
https://old.reddit.com/r/nottheonion/new.json?count=25&after=t3_cb2rhl
https://old.reddit.com/r/nottheonion/new.json?count=25&after=t3_c9smfy
https://old.reddit.com/r/nottheonion/new.json?count=25&after=t3_c8xhgq
https://old.reddit.com/r/nottheonion/new.json?count=25&after=t3_c6ot6w
https://old.reddit.com/r/nottheonion/new.json?count=25&after=t3_c4aijp
https://old.reddit.com/r/nottheonion/new.json?count=25&after=t3_c2uu3b
https://old.reddit.com/r/nottheonion/new.json?count=25&after=t3_c0yoei
https://old.reddit.com/r/nottheonion/new.json?count=25&after=t3_c0jwlq
https://old.reddit.com/r/nottheonion/new.json?count=25&after=t3_bzw545
https://old.reddit.com/r/nottheonion/new.json?count=25&after=t3_bywk7h
https://old.reddit.com/r/nottheonion/new.json?count=25&after=t3_by3mws
https://old.reddit.com/r/notthe

In [12]:
news_text = combine_text(news)

In [13]:
nto_text = combine_text(nto)

In [14]:
news_df = pd.DataFrame(news_text, columns=['text'])
news_df['is_news']= 1

In [15]:
nto_df = pd.DataFrame(nto_text, columns=['text'])
nto_df['is_news']= 0

In [16]:
reddit = pd.concat([news_df, nto_df], ignore_index=True)

In [17]:
reddit['is_news'].value_counts()

0    4335
1    4310
Name: is_news, dtype: int64

In [18]:
reddit.shape

(8645, 2)

In [19]:
reddit = reddit.drop_duplicates()

In [20]:
reddit.shape

(3073, 2)

In [21]:
reddit['is_news'].value_counts(normalize=True)

0    0.592255
1    0.407745
Name: is_news, dtype: float64

In [22]:
reddit.to_csv('./datasets/reddit_3.csv', index=False)